# Proyecto Inteligencia de Negocio con Procesos ETL

## Practica 1
## Grupo 11:
  - Malave Yela Roberto
  - Silva Naranjo Bryan Patricio

### Los datos fueron extraidos de la siguiente fuente de Kaggle:
 https://www.kaggle.com/datasets/computingvictor/transactions-fraud-datasets?resource=download

 #### En el proyecto se utilizaron 2 archivos CSV, un archivo JSON y un archivo SQL que originalmente era csv pero se lo transformó a sql

### Este dataset resulta útil para el análisis, ya que permite:

- Explorar patrones de comportamiento financiero en usuarios legítimos frente a usuarios fraudulentos.

- Identificar relaciones entre el tipo de transacción y la probabilidad de fraude.

- Analizar tendencias temporales (momentos del día o secuencia de transacciones en que ocurre el fraude).

- Construir modelos predictivos y evaluar el impacto de sus distintas variables estadísticas

# Instalacion del Contenedor de Docker y la base de Datos

### Desde la consola de docker desktop ejecutamos los siguientes comandos, estos crearan el contenedor que aloja a una Base de Datos Postgress llamada db_grupo11

docker pull postgres

docker run --name cont_int_grupo11 -e POSTGRES_USER=admin -e POSTGRES_PASSWORD=adminpass -e POSTGRES_DB=db_grupo11 -p 5432:5432 -d postgres

# Instalación de paquetes del ambiente virtual

In [14]:
#!pip install pandas dotenv sqlalchemy
#!pip install psycopg2

# Importación de Dependencias


In [20]:
import pandas as pd
from dotenv import load_dotenv
import os
load_dotenv()
from sqlalchemy import create_engine

## Leer Variables de Entorno de la DataBase

Se imprime una variable para comprobar su funcionamiento

In [21]:
DB_USER=os.getenv('DB_USER')
DB_PASS=os.getenv('DB_PASS')
DB_NAME=os.getenv('DB_NAME')
DB_HOST=os.getenv('DB_HOST')

print(DB_HOST)

localhost


## Previo al siguiente apartado, se crea la conexión con la base de datos postgres desde DataSPell

###  Cargar Base de Datos
### Se crea el engine o controlador que se conecta a la base de datos

In [26]:
engine=create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')

# Generación de los DataFrame
## Carga de Datos Json y CSV

In [37]:
df_fraud=pd.read_json('data/train_fraud_labels.json')
df_cards=pd.read_csv('data/cards_data.csv')
df_transactions=pd.read_csv('data/transactions_data.csv')
df_users=pd.read_csv('data/users_data.csv')

# Conversión de CSV a SQL del dataSet Users
### Después se agrega una tabla SQL

In [49]:
df_users.to_sql('users', engine, if_exists='replace', index=False)

1000

### Se ejecuta un comando SQL para verificar que la tabla quedo registrada

In [29]:
df_users_sql=pd.read_sql('select * from users', engine)
#df_users_sql

# Visualización y Filtros de los DataFrame
## DataFrame 1



In [38]:
df_fraud.head(10)

,target
10649266,No
23410063,No
9316588,No
12478022,No
9558530,No
12532830,No
19526714,No
9906964,No
13224888,No
13749094,No


In [39]:

df_fraud.describe()

,target
count,8914963
unique,2
top,No
freq,8901631


## DataFrame 2

In [40]:
df_cards.head(10)

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No
5,4537,1746,Visa,Credit,4404898874682993,09/2003,736,YES,1,$27500,09/2003,2012,No
6,1278,1746,Visa,Debit,4001482973848631,07/2022,972,YES,2,$28508,02/2011,2011,No
7,3687,1746,Mastercard,Debit,5627220683410948,06/2022,48,YES,2,$9022,07/2003,2015,No
8,3465,1746,Mastercard,Debit (Prepaid),5711382187309326,11/2020,722,YES,2,$54,06/2010,2015,No
9,3754,1746,Mastercard,Debit (Prepaid),5766121508358701,02/2023,908,YES,1,$99,07/2006,2012,No


In [41]:
df_cards.describe()

,id,client_id,card_number,cvv,num_cards_issued,year_pin_last_changed
count,6146.000000,6146.000000,6.146000e+03,6146.000000,6146.000000,6146.000000
mean,3072.500000,994.939636,4.820426e+15,506.220794,1.503091,2013.436707
std,1774.341709,578.614626,1.328582e+15,289.431123,0.519191,4.270699
min,0.000000,0.000000,3.001055e+14,0.000000,1.000000,2002.000000
25%,1536.250000,492.250000,4.486365e+15,257.000000,1.000000,2010.000000
50%,3072.500000,992.000000,5.108957e+15,516.500000,1.000000,2013.000000
75%,4608.750000,1495.000000,5.585237e+15,756.000000,2.000000,2017.000000
max,6145.000000,1999.000000,6.997197e+15,999.000000,3.000000,2020.000000


Filtro para mostrar cuantas tarjetas tiene cada cliente

In [42]:
df_cards.groupby("client_id")["id"].count()

client_id
0       4
1       3
2       5
3       4
4       5
       ..
1995    4
1996    3
1997    7
1998    3
1999    2
Name: id, Length: 2000, dtype: int64

Filtro que muestra el limite de credito de mayor a menor

In [44]:
df_cards.sort_values("credit_limit", ascending=False)

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
2773,1026,743,Visa,Debit,4251505296439839,11/2023,630,YES,1,$9998,02/2003,2010,No
694,476,1804,Mastercard,Debit,5979460179212685,10/2022,565,YES,1,$9984,01/2020,2020,No
903,487,1424,Mastercard,Debit,5004994096233324,03/2020,489,NO,1,$9957,01/2020,2020,No
6106,2285,97,Mastercard,Debit,5447193146031175,12/2023,290,YES,2,$9956,03/2011,2011,No
3001,3746,1475,Visa,Debit,4818828811526445,05/2024,311,YES,2,$9956,07/2005,2010,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,3443,846,Visa,Credit,4518067619451768,06/2020,429,YES,2,$0,06/2009,2011,No
478,5957,1975,Mastercard,Credit,5320022308833354,12/2021,92,YES,1,$0,12/2009,2010,No
4633,265,37,Discover,Credit,6845375674595536,02/2024,943,YES,1,$0,01/2011,2011,No
221,4318,668,Mastercard,Credit,5764603958082866,08/2021,397,YES,1,$0,08/2010,2010,No


## DataFrame 3

In [45]:
df_transactions.head(10)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN
5,7475333,2010-01-01 00:07:00,1807,165,$4.81,Swipe Transaction,20519,Bronx,NY,10464.0,5942,NaN
6,7475334,2010-01-01 00:09:00,1556,2972,$77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
7,7475335,2010-01-01 00:14:00,1684,2140,$26.46,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN
8,7475336,2010-01-01 00:21:00,335,5131,$261.58,Online Transaction,50292,ONLINE,NaN,NaN,7801,NaN
9,7475337,2010-01-01 00:21:00,351,1112,$10.74,Swipe Transaction,3864,Flushing,NY,11355.0,5813,NaN


Eliminar columna errors ya que es irrelevante, todos sus datos son NaN

In [46]:
df_transactions.drop("errors", axis=1, inplace=False)


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813
...,...,...,...,...,...,...,...,...,...,...,...
13305910,23761868,2019-10-31 23:56:00,1718,2379,$1.11,Chip Transaction,86438,West Covina,CA,91792.0,5499
13305911,23761869,2019-10-31 23:56:00,1766,2066,$12.80,Online Transaction,39261,ONLINE,NaN,NaN,5815
13305912,23761870,2019-10-31 23:57:00,199,1031,$40.44,Swipe Transaction,2925,Allen,TX,75002.0,4900
13305913,23761873,2019-10-31 23:58:00,1986,5443,$4.00,Chip Transaction,46284,Daly City,CA,94014.0,5411


Filtro para saber la transaccion con mayor cantidad de dinero

In [47]:
maximo = df_transactions["amount"].max()
minimo = df_transactions["amount"].min()
print("Mayor:", maximo, "Menor:", minimo)


Mayor: $999.97 Menor: $-0.00


## DataFrame 4

In [50]:
df_users.head(10)

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1
5,68,42,70,1977,10,Male,58 Birch Lane,41.55,-90.60,$20599,$41997,$0,704,3
6,1075,36,67,1983,12,Female,5695 Fifth Street,38.22,-85.74,$25258,$51500,$102286,672,3
7,1711,26,67,1993,12,Male,1941 Ninth Street,45.51,-122.64,$26790,$54623,$114711,728,1
8,1116,81,66,1938,7,Female,11 Spruce Avenue,40.32,-75.32,$26273,$42509,$2895,755,5
9,1752,34,60,1986,1,Female,887 Grant Street,29.97,-92.12,$18730,$38190,$81262,810,1


Filtro para saber cuantos usuarios son hombres y mujeres

In [51]:
df_users["gender"].value_counts()

gender
Female    1016
Male       984
Name: count, dtype: int64

Filtro para saber el usuarios con mayor puntaje de credito

In [52]:
df_users.loc[df_users["credit_score"].idxmax()]

id                                1884
current_age                         18
retirement_age                      64
birth_year                        2001
birth_month                          5
gender                            Male
address              660 Seventh Drive
latitude                         39.98
longitude                       -82.98
per_capita_income               $28092
yearly_income                   $57281
total_debt                      $89114
credit_score                       850
num_credit_cards                     1
Name: 30, dtype: object

In [54]:
maximo = df_users["credit_score"].max()
minimo = df_users["credit_score"].min()

print("Mayor puntaje:", max_score)
print("Menor puntaje:", min_score)

Mayor puntaje: 850
Menor puntaje: 480
